In [ ]:
!pip install reverse_geocoder
!pip install geocoder
!pip install folium


In [ ]:
import pandas as pd
import zipfile
import reverse_geocoder as rg 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
sns.set()
import types
from botocore.client import Config
import folium
from geopy.geocoders import Nominatim

zf = zipfile.ZipFile('/content/test.zip') 
df_test = pd.read_csv(zf.open('test.csv'))

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df_test

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N
...,...,...,...,...,...,...,...,...,...
625129,id3008929,1,2016-01-01 00:02:52,1,-74.003464,40.725105,-74.001251,40.733643,N
625130,id3700764,1,2016-01-01 00:01:52,1,-74.006363,40.743782,-73.953407,40.782467,N
625131,id2568735,1,2016-01-01 00:01:24,2,-73.972267,40.759865,-73.876602,40.748665,N
625132,id1384355,1,2016-01-01 00:00:28,1,-73.976501,40.733562,-73.854263,40.891788,N


In [ ]:
df_test['pickup_datetime']=pd.to_datetime(df_test['pickup_datetime'])


df_test['pickup_day']=df_test['pickup_datetime'].dt.day_name()
df_test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_day
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N,Thursday
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N,Thursday
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N,Thursday
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N,Thursday
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N,Thursday


In [ ]:
df_test.insert(loc = 10,column = 'pickup_location',value = 'N/A')
df_test.insert(loc = 11,column = 'dropoff_location',value='N/A')
df_test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_day,pickup_location,dropoff_location
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N,Thursday,N/A,N/A
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N,Thursday,N/A,N/A
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N,Thursday,N/A,N/A
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N,Thursday,N/A,N/A
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N,Thursday,N/A,N/A


In [ ]:
df_test1 = df_test.sample(1000)
df_test1.reset_index(inplace=True)
df_test1.head()

,index,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_day,pickup_location,dropoff_location
0,153584,id1844340,2,2016-05-16 04:08:05,1,-73.971687,40.750500,-73.985970,40.757591,N,Monday,N/A,N/A
1,596946,id1283331,1,2016-01-09 23:55:58,1,-73.956406,40.771408,-73.958832,40.763779,N,Saturday,N/A,N/A
2,178795,id1899427,1,2016-05-08 22:52:04,1,-73.989502,40.741306,-73.988159,40.756496,N,Sunday,N/A,N/A
3,34861,id2758094,2,2016-06-20 12:35:56,2,-73.952545,40.780846,-73.964653,40.791828,N,Monday,N/A,N/A
4,454817,id3379567,2,2016-02-21 13:44:03,1,-73.991257,40.750408,-74.005844,40.740280,N,Sunday,N/A,N/A


In [ ]:
for i in range(len(df_test1)):
  print(i)
  df_test1['pickup_location'][i]=rg.search((df_test1['pickup_latitude'][i],df_test1['pickup_longitude'][i]))[0]['name']
  df_test1['dropoff_location'][i]=rg.search((df_test1['dropoff_latitude'][i],df_test1['dropoff_longitude'][i]))[0]['name']

0
Loading formatted geocoded file...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [ ]:
from google.colab import files
df_test1.to_csv('test.csv') 
files.download('test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_test1

,index,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_day,pickup_location,dropoff_location
0,153584,id1844340,2,2016-05-16 04:08:05,1,-73.971687,40.750500,-73.985970,40.757591,N,Monday,Long Island City,Manhattan
1,596946,id1283331,1,2016-01-09 23:55:58,1,-73.956406,40.771408,-73.958832,40.763779,N,Saturday,Manhattan,Manhattan
2,178795,id1899427,1,2016-05-08 22:52:04,1,-73.989502,40.741306,-73.988159,40.756496,N,Sunday,New York City,Manhattan
3,34861,id2758094,2,2016-06-20 12:35:56,2,-73.952545,40.780846,-73.964653,40.791828,N,Monday,Manhattan,Manhattan
4,454817,id3379567,2,2016-02-21 13:44:03,1,-73.991257,40.750408,-74.005844,40.740280,N,Sunday,Weehawken,New York City
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,103550,id2989007,1,2016-05-31 00:26:22,2,-74.003967,40.742088,-73.981689,40.732433,N,Tuesday,New York City,New York City
996,521964,id3851704,1,2016-02-02 19:36:08,1,-73.981506,40.767883,-73.962997,40.794079,N,Tuesday,Manhattan,Manhattan
997,23659,id3295405,1,2016-06-23 19:51:57,1,-73.994209,40.751282,-73.952065,40.777416,N,Thursday,Weehawken,Manhattan
998,34123,id0623714,2,2016-06-20 17:17:14,5,-73.965912,40.758801,-73.969223,40.753922,N,Monday,Long Island City,Long Island City


In [ ]:
df_test1

,index,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_day,pickup_location,dropoff_location
0,153584,id1844340,2,2016-05-16 04:08:05,1,-73.971687,40.750500,-73.985970,40.757591,N,Monday,Long Island City,Manhattan
1,596946,id1283331,1,2016-01-09 23:55:58,1,-73.956406,40.771408,-73.958832,40.763779,N,Saturday,Manhattan,Manhattan
2,178795,id1899427,1,2016-05-08 22:52:04,1,-73.989502,40.741306,-73.988159,40.756496,N,Sunday,New York City,Manhattan
3,34861,id2758094,2,2016-06-20 12:35:56,2,-73.952545,40.780846,-73.964653,40.791828,N,Monday,Manhattan,Manhattan
4,454817,id3379567,2,2016-02-21 13:44:03,1,-73.991257,40.750408,-74.005844,40.740280,N,Sunday,Weehawken,New York City
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,103550,id2989007,1,2016-05-31 00:26:22,2,-74.003967,40.742088,-73.981689,40.732433,N,Tuesday,New York City,New York City
996,521964,id3851704,1,2016-02-02 19:36:08,1,-73.981506,40.767883,-73.962997,40.794079,N,Tuesday,Manhattan,Manhattan
997,23659,id3295405,1,2016-06-23 19:51:57,1,-73.994209,40.751282,-73.952065,40.777416,N,Thursday,Weehawken,Manhattan
998,34123,id0623714,2,2016-06-20 17:17:14,5,-73.965912,40.758801,-73.969223,40.753922,N,Monday,Long Island City,Long Island City
